### Evaluating Recommendation models on all the data

In [1]:

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_836eb90819e64bfab21841eef5b3bf3c(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_6429e107b4727d3f5906de019f7fad88ada04fb8','domain': {'id': 'b5bbfbbc2d764a9096fd4aeb7d0d1477'},
            'password': 'o{a{0]Dnx^MJ-dc1'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)



In [2]:
normratings= pd.read_csv(get_object_storage_file_with_credentials_836eb90819e64bfab21841eef5b3bf3c('insta', 'recommendwithratings.csv'))
normratings.head()


,user_id,product_id,rating
0,1,196,5.000000
1,1,10258,4.545455
2,1,12427,4.545455
3,1,25133,4.090909
4,1,13032,1.818182


In [3]:
!pip install --user --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/goel.t@husky.neu.edu/E914-C086-1338-7082-B278-E8A7-1C13-1FDE/GraphLab-Create-License.tar.gz
    

Requirement already up-to-date: graphlab-create>=2.1 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from GraphLab-Create-License==2.1)
Requirement already up-to-date: psclient in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: boto==2.33.0 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: decorator==4.0.9 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sed1-980fc88e6d89d3-f770736419c3/.local/lib/python2.7/site-packages (from graphlab-create>=2.1->GraphLab-Create-License==2.1)
Requirement already up-to-date: tornado==4.3 in /usr/local/src/bluemix_jupyter_bundle.v73/notebook/lib/python2.7/site-packages (from g

In [4]:
import numpy as np
import graphlab

msk = np.random.rand(len(normratings)) < 0.8
train = normratings[msk]
test = normratings[~msk]
train_data = graphlab.SFrame(train)
test_data=graphlab.SFrame(test)

This non-commercial license of GraphLab Create for academic use is assigned to goel.t@husky.neu.edu and will expire on December 04, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1513103137.log


#### Creating item similarity model, popularity model and latent model on the whole data for comparing and selecting best out of them

In [5]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating', similarity_type='pearson')
popularity_model = graphlab.popularity_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating')
latent_model=graphlab.factorization_recommender.create(train_data, user_id='user_id', item_id='product_id', target='rating')
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model,latent_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model,latent_model])

Recsys training: model = item_similarity

Preparing data set.

Data has 11091223 observations with 206125 users and 49636 items.

Data prepared in: 11.3874s

Training model from provided data.

...

recommendations finished on 199000/202029 queries. users per second: 2880.9

recommendations finished on 200000/202029 queries. users per second: 2882.64

recommendations finished on 201000/202029 queries. users per second: 2884.4

recommendations finished on 202000/202029 queries. users per second: 2883.57


Precision and recall summary statistics by cutoff
+--------+------------------+-------------------+
| cutoff |  mean_precision  |    mean_recall    |
+--------+------------------+-------------------+
|   1    | 0.0115626964446  | 0.000913421475807 |
|   2    | 0.0092313479748  |  0.00148233276347 |
|   3    | 0.00784375840432 |  0.00191872753452 |
|   4    | 0.00689876205891 |  0.00230404032193 |
|   5    | 0.00629513584683 |  0.00264680073719 |
|   6    | 0.00579784750374 |  0.00295004988483 |
|   7    | 0.00543910598408 |  0.00326441402051 |
|   8    | 0.00510694009276 |  0.00352964737372 |
|   9    | 0.00484253910742 |  0.00379207555727 |
|   10   | 0.0046156739874  |  0.00403881432145 |
+--------+------------------+-------------------+
[10 rows x 3 columns]

Model compare metric: precision_recall
Canvas is accessible via web browser at the URL: http://localhost:40469/index.html
Opening Canvas in default web browser.


###  As we can see that latent model which uses unsupervised learning to shrink the users traits into latent features gives us the best result

###  Below is the code that is happening in the library
  product_crosstab=pd.pivot_table(data=normratings,values='rating',index='user_id',columns='product_id',fill_value=0)

  X=product_crosstab.values.T
  
  import sklearn
  
  from sklearn.decomposition import TruncatedSVD

  SVD=TruncatedSVD(n_components=20,random_state=17)

  resultant_matrix=SVD.fit_transform(X)  
  
  resultant_matrix.shape
  
  import numpy as np
  
  corr_mat = np.corrcoef(resultant_matrix)
   
  product_ids=product_crosstab.columns
  
  productlist=list(product_ids)
  
  productindexlist=[]
  
 for i in productlist:
 
    productindexlist.append(productlist.index(i))
    
  allrecommendations=[]    
  
  for j in productindexlist:
  
    corr_j=corr_mat[j]
 
    corrlist=list(product_ids[(corr_j<1.0)&(corr_j>0.98)])
    
    corrlist.append(j)
  
    allrecommendations.append(corrlist)
  
  print(allrecommendations)

### We use the Graphlab because it was able to scale since it was able to utilize the ibm dsx spark clusters